**전체**

신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라고 한다. 
신경망은 다음과 같이 4단계로 수행된다.

**1단계** - **미니배치**  
- 훈련 데이터 중 일부를 무작위로 가져온다. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실 함수 값을 줄이는 것이 목표이다.

**2단계** - **기울기 산출**  
- 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시한다.

**3단계** - **매개변수 갱신**  
- 가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다.

**4단계** - **반복**  
- 1~3단계를 반복한다.


이는 경사 하강법으로 매개변수를 갱신하는 방법이며, 이때 데이터를 미니배치로 무작위로 선정하기 때문에 **확률적 경사 하강법**이라고 부른다.

### 4.5.1 2층 신경망 클래스 구현하기

In [4]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from common.functions import *
from common.gradient import numerical_gradient


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads

#### [표 4-1]  TwoLayerNet 클래스가 사용하는 변수

|**변수**|**설명**|
|:-----:|:-----:|
|**params**|신경망의 매개변수를 보관하는 딕셔너리 변수(인스턴스 변수)|
|-|params['W1']은 1번째 층의 가중치, params['b1']은 1번째 층의 편향|
|-|params['W2']은 2번째 층의 가중치, params['b2']은 2번째 층의 편향|
|**grads**|기울기 보관하는 딕셔너리 변수(numerical_gradient()메서드의 반환값)|
|-|grads['W1']은 1번째 층의 가중치의 기울기, grads['b1']은 1번째 층의 편향의 기울기|
|-|grads['W2']은 1번째 층의 가중치의 기울기, grads['b2']은 1번째 층의 편향의 기울기|

#### [표 4-2] TwoLayerNet 클래스 메서드
|**메서드**|**설명**|
|:-----:|:-----:|
|**`__init__(self, input_size, hidden_size, output_size)`**| 초기화를 수행한다. |
| - |입력층의 뉴런의 수, 은닉층의 뉴런 수, 출력층의 뉴런 수 |
|**`predict(self, x)`**|예측(추론)을 수행한다.|
| - | x는 입력 데이터|
|**`loss(self, x, t)`**|손실 함수의 값을 구한다.|
| - | x = 입력데이터, t = 정답 레이블|
| **`accuracy(self, x, t)`** | 정확도를 구한다. |
| **`numerical_gradient(self, x, t)`** | 가중치 매개변수의 기울기를 구한다.|
| **`gradient(self, x, t)`** | 가중치 매개변수의 기울기를 구한다.|
| - | numerical_gradient(self, x, t)의 개선판!|
| - | 구현은 다음장 | 

#### 신경망의 가중치(매개변수)를 보관하는 **`params`**  변수를 확인

In [8]:
net = TwoLayerNet(input_size = 784, hidden_size = 100, output_size = 10)
print(net.params['W1'].shape)
print(net.params['b1'].shape)
print(net.params['W2'].shape)
print(net.params['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


params 변수의 역할 : 신경망의 모든 매개변수가 저장. 그리고 변수에 저장된 가중치 매개변수가 예측 처리(순방향 처리)에서 사용된다.

In [12]:
x = np.random.rand(100, 784) # 더미 입력 데이터(이미지 100장 분량)
y = net.predict(x)

#### 신경망의 가중치의 기울기를 보관하는  **`grads`** 변수 확인

In [14]:
x = np.random.rand(100, 784) # 더미 입력 데이터(이미지 100장 분량)
t = np.random.rand(100, 10)  # 더미 정답 레이블 (100장 분량)

grads = net.numerical_gradient(x, t) # 기울기 계산

print(grads['W1'].shape)
print(grads['b1'].shape)
print(grads['W2'].shape)
print(grads['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)
